In [ ]:
import requests

# Replace with your own values
access_token = 'YOUR_ACCESS_TOKEN'
instagram_account_id = 'INSTAGRAM_ACCOUNT_ID'

# API endpoint URLs
base_url = 'https://graph.instagram.com/v17.0/'
media_url = f'{base_url}{instagram_account_id}/media'
insights_url = f'{base_url}{instagram_account_id}/insights'

# Parameters for the API requests
media_params = {
    'fields': 'id,media_type,media_url,caption,timestamp',
    'access_token': access_token
}

insights_params = {
    'metric': 'impressions,reach,engagement',
    'period': 'day',
    'access_token': access_token
}

# Function to fetch and print media data
def get_media_data():
    response = requests.get(media_url, params=media_params)
    data = response.json()
    for media in data['data']:
        print(f"Media ID: {media['id']}")
        print(f"Media Type: {media['media_type']}")
        print(f"Media URL: {media['media_url']}")
        print(f"Caption: {media.get('caption', 'No caption')}")
        print(f"Timestamp: {media['timestamp']}")
        print("--------------------------")

# Function to fetch and print insights data
def get_insights_data():
    response = requests.get(insights_url, params=insights_params)
    data = response.json()
    for insight in data['data']:
        print(f"Date: {insight['period_start']}")
        for metric, value in insight['values'][0].items():
            print(f"{metric.capitalize()}: {value}")
        print("--------------------------")

if __name__ == "__main__":
    print("Media Data:")
    get_media_data()

    print("\nInsights Data:")
    get_insights_data()
